# Resistance/Support AND Candles Patterns

In [2]:
import pandas as pd
import MetaTrader5 as mt5

In [15]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    
ativo = 'WIN@N'
df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M5, 0, 500))
df['Data'] = pd.to_datetime(df['time'], unit='s').apply(lambda x: str(x))
df = df[['Data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df = df.rename(columns = {'real_volume':'vol'})
df

,Data,Open,High,Low,Close,Vol
0,2023-01-09 14:25:00,110825.0,111075.0,110810.0,111010.0,115990
1,2023-01-09 14:30:00,111015.0,111015.0,110765.0,110835.0,120860
2,2023-01-09 14:35:00,110835.0,110980.0,110760.0,110935.0,105373
3,2023-01-09 14:40:00,110935.0,111000.0,110750.0,110805.0,120920
4,2023-01-09 14:45:00,110810.0,110985.0,110760.0,110955.0,120418
...,...,...,...,...,...,...
495,2023-01-13 18:00:00,111990.0,112035.0,111975.0,111995.0,35028
496,2023-01-13 18:05:00,112000.0,112220.0,111995.0,112185.0,74534
497,2023-01-13 18:10:00,112180.0,112195.0,112065.0,112090.0,52773
498,2023-01-13 18:15:00,112095.0,112100.0,111980.0,112015.0,34915


# Support and Resistance FUNCTIONS

In [16]:
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.Low[i]>df1.Low[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.Low[i]<df1.Low[i-1]):
            return 0
    return 1

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.High[i]<df1.High[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.High[i]>df1.High[i-1]):
            return 0
    return 1

In [17]:
length = len(df)
High = list(df['High'])
Low = list(df['Low'])
Close = list(df['Close'])
Open = list(df['Open'])
bodydiff = [0] * length

Highdiff = [0] * length
Lowdiff = [0] * length
ratio1 = [0] * length
ratio2 = [0] * length

def isEngulfing(l):
    row=l
    bodydiff[row] = abs(Open[row]-Close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001      

    bodydiffmin = 0.002
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        Open[row-1]<Close[row-1] and
        Open[row]>Close[row] and 
        (Open[row]-Close[row-1])>=-0e-5 and Close[row]<Open[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        Open[row-1]>Close[row-1] and
        Open[row]<Close[row] and 
        (Open[row]-Close[row-1])<=+0e-5 and Close[row]>Open[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0
       
def isStar(l):
    bodydiffmin = 0.0020
    row=l
    Highdiff[row] = High[row]-max(Open[row],Close[row])
    Lowdiff[row] = min(Open[row],Close[row])-Low[row]
    bodydiff[row] = abs(Open[row]-Close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001
    ratio1[row] = Highdiff[row]/bodydiff[row]
    ratio2[row] = Lowdiff[row]/bodydiff[row]

    if (ratio1[row]>1 and Lowdiff[row]<0.2*Highdiff[row] and bodydiff[row]>bodydiffmin):# and Open[row]>Close[row]):
        return 1
    elif (ratio2[row]>1 and Highdiff[row]<0.2*Lowdiff[row] and bodydiff[row]>bodydiffmin):# and Open[row]<Close[row]):
        return 2
    else:
        return 0
    
def CloseResistance(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.High[l]-min(levels, key=lambda x:abs(x-df.High[l])))<=lim
    c2 = abs(max(df.Open[l],df.Close[l])-min(levels, key=lambda x:abs(x-df.High[l])))<=lim
    c3 = min(df.Open[l],df.Close[l])<min(levels, key=lambda x:abs(x-df.High[l]))
    c4 = df.Low[l]<min(levels, key=lambda x:abs(x-df.High[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0
    
def CloseSupport(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.Low[l]-min(levels, key=lambda x:abs(x-df.Low[l])))<=lim
    c2 = abs(min(df.Open[l],df.Close[l])-min(levels, key=lambda x:abs(x-df.Low[l])))<=lim
    c3 = max(df.Open[l],df.Close[l])>min(levels, key=lambda x:abs(x-df.Low[l]))
    c4 = df.High[l]>min(levels, key=lambda x:abs(x-df.Low[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0

In [18]:
n1=1
n2=0
backCandles=30
signal = [0] * length

for row in range(backCandles, len(df)-n2):
    ss = []
    rr = []
    for subrow in range(row-backCandles+n1, row+1):
        if support(df, subrow, n1, n2):
            ss.append(df.Low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.High[subrow])
    '''#!!!! parameters
    if ((isEngulfing(row)==1 or isStar(row)==1) and CloseResistance(row, rr, 150e-5) ):#and df.RSI[row]<30
        signal[row] = 1
    elif((isEngulfing(row)==2 or isStar(row)==2) and CloseSupport(row, ss, 150e-5)):#and df.RSI[row]>70
        signal[row] = 2
    else:
        signal[row] = 0'''

    if  CloseResistance(row, rr, 150e-5):#and df.RSI[row]<30
        signal[row] = 1
    elif CloseSupport(row, ss, 150e-5):#and df.RSI[row]>70
        signal[row] = 2
    else:
        signal[row] = 0


tempo_lista =10
for num, i in enumerate(signal):
    #print(num,i)
    if i != 0:
        for i in range(tempo_lista):
            try:
                n_num = num + i + 1
                signal[n_num] = 0
            except:
                pass

In [19]:
df['signal']=signal

#df['signal'] = df['signal'].shift(1)

In [20]:
df[df['signal'] != 0].tail(10)

,Data,Open,High,Low,Close,Vol,signal
389,2023-01-13 09:10:00,112700.0,112830.0,112655.0,112780.0,101915,2
400,2023-01-13 10:05:00,111890.0,111920.0,111585.0,111665.0,275973,1
411,2023-01-13 11:00:00,111965.0,111985.0,111730.0,111755.0,205428,2
422,2023-01-13 11:55:00,112290.0,112325.0,112110.0,112160.0,159143,1
433,2023-01-13 12:50:00,112075.0,112180.0,112070.0,112105.0,86989,1
444,2023-01-13 13:45:00,112095.0,112130.0,112045.0,112115.0,81345,1
456,2023-01-13 14:45:00,112050.0,112135.0,112020.0,112085.0,86211,1
467,2023-01-13 15:40:00,112165.0,112330.0,112165.0,112280.0,112495,1
478,2023-01-13 16:35:00,112055.0,112090.0,111955.0,112025.0,110195,2
489,2023-01-13 17:30:00,112155.0,112190.0,112065.0,112120.0,90132,1


In [21]:
df[df['signal']==2].count()

Data      15
Open      15
High      15
Low       15
Close     15
Vol       15
signal    15
dtype: int64

In [22]:
def SIGNAL():
    return df.signal

In [188]:
#A new strategy needs to extend Strategy class and override its two abstract methods: init() and next().
#Method init() is invoked before the strategy is run. Within it, one ideally precomputes in efficient, 
#vectorized manner whatever indicators and signals the strategy depends on.
#Method next() is then iteratively called by the Backtest instance, once for each data point (data frame row), 
#simulating the incremental availability of each new full candlestick bar.

#Note, backtesting.py cannot make decisions / trades within candlesticks — any new orders are executed on the
#next candle's Open (or the current candle's Close if trade_on_Close=True). 
#If you find yourself wishing to trade within candlesticks (e.g. daytrading), you instead need to begin 
#with more fine-grained (e.g. hourly) data.

In [23]:
df['Data'] = pd.to_datetime(df['Data'])
df = df.set_index('Data')

In [24]:
df

,Open,High,Low,Close,Vol,signal
Data,,,,,,
2023-01-09 14:25:00,110825.0,111075.0,110810.0,111010.0,115990,0
2023-01-09 14:30:00,111015.0,111015.0,110765.0,110835.0,120860,0
2023-01-09 14:35:00,110835.0,110980.0,110760.0,110935.0,105373,0
2023-01-09 14:40:00,110935.0,111000.0,110750.0,110805.0,120920,0
2023-01-09 14:45:00,110810.0,110985.0,110760.0,110955.0,120418,0
...,...,...,...,...,...,...
2023-01-13 18:00:00,111990.0,112035.0,111975.0,111995.0,35028,0
2023-01-13 18:05:00,112000.0,112220.0,111995.0,112185.0,74534,0
2023-01-13 18:10:00,112180.0,112195.0,112065.0,112090.0,52773,0


In [25]:
from backtesting import Strategy, Backtest

class MyCandlesStrat(Strategy):  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        sl = 100
        tp = 200
        if self.signal1==2:
            sl1 = self.data.Close[-1] - sl
            tp1 = self.data.Close[-1] + tp
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1:
            sl1 = self.data.Close[-1] + sl
            tp1 = self.data.Close[-1] - tp
            self.sell(sl=sl1, tp=tp1)

bt = Backtest(df, MyCandlesStrat, cash=100000000, commission=0)
bt.run()


Start                     2023-01-09 14:25:00
End                       2023-01-13 18:20:00
Duration                      4 days 03:55:00
Exposure Time [%]                        18.0
Equity Final [$]                   99760140.0
Equity Peak [$]                   100356990.0
Return [%]                           -0.23986
Buy & Hold Return [%]                0.882803
Return (Ann.) [%]                  -15.763375
Volatility (Ann.) [%]                5.070622
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.872495
Avg. Drawdown [%]                   -0.300497
Max. Drawdown Duration        2 days 07:05:00
Avg. Drawdown Duration        0 days 13:02:00
# Trades                                   42
Win Rate [%]                        30.952381
Best Trade [%]                       0.186406
Worst Trade [%]                     -0.095463
Avg. Trade [%]                    

In [190]:
#fixed distance Trailing SL
from backtesting import Strategy, Backtest
import numpy as np

class MyCandlesStrat(Strategy):
    sltr=200
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        sltr = self.sltr
        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - sltr)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + sltr) 

        if self.signal1==2 and len(self.trades)==0: # trades number change!
            sl1 = self.data.Close[-1] - sltr
            self.buy(sl=sl1)
        elif self.signal1==1 and len(self.trades)==0: # trades number change!
            sl1 = self.data.Close[-1] + sltr
            self.sell(sl=sl1)


bt = Backtest(df, MyCandlesStrat, cash=100000000, commission=0)
stat = bt.run()
stat

C:\Users\Usuário\AppData\Local\Temp\ipykernel_8240\4108643265.py:28: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, MyCandlesStrat, cash=100000000, commission=0)


Start                                     0.0
End                                    4999.0
Duration                               4999.0
Exposure Time [%]                        40.1
Equity Final [$]                   92889200.0
Equity Peak [$]                   101644655.0
Return [%]                            -7.1108
Buy & Hold Return [%]                1.943471
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -10.755302
Avg. Drawdown [%]                   -3.008073
Max. Drawdown Duration                 4829.0
Avg. Drawdown Duration                 1240.0
# Trades                                401.0
Win Rate [%]                        33.416459
Best Trade [%]                       2.078146
Worst Trade [%]                     -1.427745
Avg. Trade [%]                    

In [27]:
bt.plot(show_legend=False,plot_equity=False)

Row(id='1901', ...)